In [ ]:
# !pip install --upgrade pip
# !pip install gensim
# !pip install nltk
# !pip install tokenizers
# !pip install sentencepiece
# !pip install python-bidi
# !pip install arabic-reshaper
# !pip install PyArabic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
# from sklearn.metrics import accuracy_score
import numpy as np
from utilities import *
import textProcessing as tp
import nltk
import data_preprocessing as dp
from torch.optim.lr_scheduler import StepLR
# # nltk.download('punkt')
# train_text = load_text("dataset/train.txt")
# tp.preprocessing_text(train_text,"train_preprocessed.txt")


{'ً': 0, 'ٌ': 1, 'ٍ': 2, 'َ': 3, 'ُ': 4, 'ِ': 5, 'ّ': 6, 'ًّ': 7, 'ٌّ': 8, 'ٍّ': 9, 'َّ': 10, 'ُّ': 11, 'ِّ': 12, 'ْ': 13, '': 14}
15
{'ء': 0, 'آ': 1, 'أ': 2, 'ؤ': 3, 'إ': 4, 'ئ': 5, 'ا': 6, 'ب': 7, 'ة': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ى': 34, 'ي': 35, ' ': 36, '0': 37, 's': 38}
39
قوله أو قطع الأول يده إلخ قال الزركشي
sentence قَوْلُهُ أَوْ قَطَعَ الأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ
reference قَوْلُهُ أَوْ قَطَعَ الأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ


In [18]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = load_text("dataset/train_preprocessed.txt")
        self.transform = ToTensor()
        self.T = 100

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sentence = self.data[idx]
        # extract the label
        labels,sentence= tp.extract_diacritics_with_previous_letter('s'+sentence)

        # remove the label from the sentence
        # sentence = tp.clear_diacritics(sentence)

        # convert the sentence to one hot encoding
        sentence = convert_sentence_to_vector(sentence)

        # convert the labels to one hot encoding
        labels = convert_labels_to_vector(labels)

        # pad the sentence and labels if smaller than T
        temp = np.zeros((self.T, 39))
        if self.T > sentence.shape[0]:
            temp[:sentence.shape[0], :] = sentence
            # append the sentence with spaces
            # temp[sentence.shape[0]:, 36] = 1
            sentence = temp
        else:
            sentence = sentence[:self.T, :]
        

        temp = np.zeros((self.T,15))
        if self.T > labels.shape[0]:
            temp[:labels.shape[0], :] = labels
            labels = temp
        else:
            labels = labels[:self.T,:]
            # append the labels with no diacritics
            # labels[self.T:, 14] = 1 

        #-----------------------------------------
        # reshape the labels
            
        # labels = labels.reshape(-1,1)
        # temp = np.zeros((self.T,1))
        # if self.T > labels.shape[0]:
        #     temp[:labels.shape[0], :] = labels
        #     labels = temp
        # else:
        #     labels = labels[:self.T]
        #     # append the labels with no diacritics
        #     # labels[self.T:, 14] = 1 

        # convert the sentence and labels to tensors
        sentence = torch.tensor(sentence, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32)
        # labels = torch.LongTensor(labels)

        return sentence, labels


In [19]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 2
        # define the embedding layer
        # self.embedding = nn.Embedding(input_size, hidden_size)
        # self.rnn = nn.RNN(hidden_size, hidden_size,num_layers=self.num_layers, batch_first=True)

        self.rnn = nn.RNN(self.input_size, hidden_size,num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)
        self.init_weight()

    def forward(self, input):
        # input = self.embedding(input)
        # print(input.shape)
        output, _ = self.rnn(input)
        output = self.linear(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers,batch_size, self.hidden_size)
    def init_weight(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)

In [5]:
class LSTMClassifier(nn.Module):
    def _init_(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self)._init_()


        self.hidden_size = hidden_size
        self.input_size = input_size

        self.lstm = nn.LSTM(input_size, hidden_size, 5,batch_first=True)
        self.tanh = nn.Tanh()
        self.linear = nn.Linear(hidden_size, output_size)
        self.init_weight()

    def forward(self, input, hidden, c):
        output, _ = self.lstm(input, (hidden,c))
        # reshape the output to be able to pass it to the linear layer
        # output = output.contiguous().view(-1, self.hidden_size)
        output = self.tanh(output)
        output = self.linear(output)
        # output = self.soft(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)
    def init_c(self, batch_size):
        return torch.zeros(1,batch_size, self.hidden_size)
    def init_weight(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)

In [20]:
# Hyperparameter
input_size = 39
hidden_size = 128
output_size = 15
batch_size = 64
num_epochs = 100000

In [23]:
# connect to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

# Create an instance of the RNN classifier
model = RNNClassifier(input_size, hidden_size, output_size)
model.to(device)

# Load data from CSV file
dataset = MyDataset()

# Create a dataloader to handle batching and shuffling
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
# criterion = nn.CrossEntropyLoss(ignore_index=0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

cuda:0
True
1


C:\Users\ahmed sabry\AppData\Local\Temp\ipykernel_20532\2805909639.py:30: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(param)
C:\Users\ahmed sabry\AppData\Local\Temp\ipykernel_20532\2805909639.py:28: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.0)


In [24]:
# Training loop
# optimizer = optim.Adam(model.parameters(), lr=0.0005)
# model.load_state_dict(torch.load('model1.pth'))
model.train()
debug = True
for epoch in range(num_epochs):
    scheduler.step()
    for inputs, labels in train_dataloader:
            # Zero the gradients
      optimizer.zero_grad()

      # Reshape input and labels to (batch_size, seq_length, input_size)
      if debug:
        print("inputs shape",inputs.shape)
        print("labels shape",labels.shape)
        debug = False
      input = inputs.view(inputs.shape[0], -1, input_size)
      # labels = labels.view(inputs.shape[0], -1, output_size)
      input, labels = input.to(device), labels.to(device)
      # hidden = model.init_hidden(batch_size=inputs.shape[0])
      # hidden = hidden.to(device)

      # Forward pass
      output = model(input)

      # get max of outputs as index
      # output = torch.argmax(output, dim=2)
      # output = output.view(inputs.shape[0],-1, output.shape[1])

      # Compute loss
      loss = criterion(output, labels)

      # Backward pass and optimization
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
      optimizer.step()
      


      # Print loss for monitoring
      print(f"Epoch: {epoch+1}, Batch Loss: {loss.item()}")
      break
      
      



inputs shape torch.Size([64, 100, 39])
labels shape torch.Size([64, 100, 1])


d:\anaconda3\envs\pt\Lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


RuntimeError: Expected target size [64, 15], got [64, 100, 1]

In [52]:
model.train()
scheduler.step()

# optimizer = optim.Adam(model.parameters(), lr=0.0005)
for inputs, labels in train_dataloader:
    for epoch in range(num_epochs):
    # Reshape input and labels to (batch_size, seq_length, input_size)
        # input = inputs.view(inputs.shape[0], -1, input_size)
        # labels = labels.view(inputs.shape[0], -1, output_size)
        input, labels = input.to(device), labels.to(device)
        hidden = model.init_hidden(batch_size=inputs.shape[0])
        hidden = hidden.to(device)

        # Forward pass
        output = model(input, hidden)

        # Compute loss
        loss = criterion(output, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimization
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
    
        


        # Print loss for monitoring
        print(f"Epoch: {epoch+1}, Batch Loss: {loss.item()}")
    break
    
    

Epoch: 1, Batch Loss: 25.26896858215332
Epoch: 2, Batch Loss: 24.713531494140625
Epoch: 3, Batch Loss: 24.33721160888672
Epoch: 4, Batch Loss: 23.996437072753906
Epoch: 5, Batch Loss: 23.74972915649414
Epoch: 6, Batch Loss: 23.535661697387695
Epoch: 7, Batch Loss: 23.307573318481445
Epoch: 8, Batch Loss: 23.13245964050293
Epoch: 9, Batch Loss: 22.967578887939453
Epoch: 10, Batch Loss: 22.81300163269043
Epoch: 11, Batch Loss: 22.66948699951172
Epoch: 12, Batch Loss: 22.532997131347656
Epoch: 13, Batch Loss: 22.400665283203125
Epoch: 14, Batch Loss: 22.2724609375
Epoch: 15, Batch Loss: 22.150245666503906
Epoch: 16, Batch Loss: 22.0334529876709
Epoch: 17, Batch Loss: 21.91950225830078
Epoch: 18, Batch Loss: 21.80775260925293
Epoch: 19, Batch Loss: 21.699106216430664
Epoch: 20, Batch Loss: 21.594446182250977
Epoch: 21, Batch Loss: 21.493906021118164
Epoch: 22, Batch Loss: 21.397621154785156
Epoch: 23, Batch Loss: 21.305335998535156
Epoch: 24, Batch Loss: 21.21613311767578
Epoch: 25, Batch 

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model1.pth')

In [20]:
# Set the model to evaluation mode
# model.load_state_dict(torch.load('model1.pth'))
model.eval()

# Iterate over the test dataset
predictions = []
ground_truth = []
correct = 0
correct_prediction = 0
total_predictions = 0
total = 0

with torch.no_grad():
  for inputs, labels in train_dataloader:
      # Forward pass
      hidden = model.init_hidden(batch_size=inputs.shape[0])
      hidden = hidden.to(device)
      inputs ,labels = inputs.to(device),labels.to(device)
      outputs = model(inputs,hidden)
      outputs = outputs.view(inputs.shape[0], -1, output_size)
      # Get predictions from the maximum value
      _, predicted = torch.max(outputs.data, 2)
      predicted = predicted.cpu()
      labels = labels.cpu()
      # Convert to numpy arrays
      predictions.extend(predicted.numpy())

      # get labels from one hot encoding
      _, labels = torch.max(labels.data, 2)
      # Convert to numpy arrays
      ground_truth.extend(labels.numpy())
      # Compute accuracy
      correct += (predicted == labels).sum().item()
      total += labels.size(0) * labels.size(1)
      print(f"Accuracy: {correct/total}")

      
      break
      

print(f"Accuracy: {correct/total}")

# Convert lists to numpy arrays
predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

Accuracy: 0.633125
Accuracy: 0.633125


In [116]:
# clear the memory of the GPU
import gc
torch.cuda.empty_cache()
gc.collect()

1057